<a href="https://www.kaggle.com/code/nhanbaoho/european-soccer-database-with-sql?scriptVersionId=99031897" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# <center>European Soccer Database with SQL</center>

---
# **Dataset**
See <a href="https://www.kaggle.com/datasets/hugomathien/soccer">resource from Kaggle</a>

---
<a id="0"></a>
# **Table of Contents**
1. [Import packages](#1)
2. [Explore tables](#2)
3. [Analysis by country](#3)
4. [Analysis by country](#4)
5. [Analyse win and lose by teams](#5)

---
<a id='1'></a>
# **1. Import packages**
[Table of Contents](#0)

In [ ]:
from sqlite3 import connect
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

<a id='2'></a>
# **2. Explore tables**
[Table of Contents](#0)

In [ ]:
# Create a connection object
connection = connect('/kaggle/input/soccer/database.sqlite')
# connection = connect('Data/database.sqlite')

all_tables = pd.read_sql("""    
                            SELECT 
                                * 
                            FROM 
                                sqlite_master
                            WHERE 
                                type='table';
                        """, connection)

<a id='all_tables'></a>
## **2.1. Examine tables in database**

In [ ]:
all_tables

<a id='countries'></a>
## **2.2. Display all countries in a dataframe**

In [ ]:
# Display all countries in a dataframe
countries = pd.read_sql(""" 
                            SELECT   
                                * 
                            FROM 
                                Country;
                        """, connection)
countries

<a id='teams'></a>
## **2.3. Display all teams**

In [ ]:
# Display all teams
teams = pd.read_sql("""
                        SELECT 
                            * 
                        FROM 
                            Team;
                    """, connection)
teams

<a id='matches'></a>
## **2.4. Display all matches**

In [ ]:
# Display all matches
matches = pd.read_sql("""
                        SELECT 
                            * 
                        FROM 
                            Match;
                    """, connection)
matches

### **The 'Match' table has 115 columns. We will selects those we are interested in.**

* **It appear country_id and league_is is identical, let's check.**

In [ ]:
country_not_league = pd.read_sql("""
                        SELECT 
                            count(country_id) as count
                        FROM 
                            Match
                        where country_id != league_id;
                    """, connection)
country_not_league

* **We can see that the two columns are identical. We can skip one.**

### **Explore all column titles**

In [ ]:
# explore all column titles
matches_columns = matches.columns
print(list(matches_columns))

* **Select column titles of interest. Data is extracted form pandas dataframe 'matches'. See [matches](#matches)**

In [ ]:
# select columns of interest: columns' titles
selected_matches_columns = ['id', 'country_id', 'season', 'stage', 'date', 'match_api_id', 
                            'home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal']

In [ ]:
# columns of interest: of those 115 columns, we select only some
# data are retrived from dataframe
shorten_matches = matches[selected_matches_columns]
shorten_matches

### **Select column titles of interest. Data is extracted form sql table 'Match'**

In [ ]:
# We can perform the same task using sql query
# Display all matches. 
matches = pd.read_sql("""
                        SELECT 
                            id, country_id, season, stage, date, match_api_id, 
                            home_team_api_id, away_team_api_id, home_team_goal, away_team_goal
                        FROM 
                            Match;
                    """, connection)
matches

<a id = 'players'></a>
## **2.6.  Display all players**

In [ ]:
# Display all players
players = pd.read_sql("""
                        SELECT 
                            * 
                        FROM 
                            Player;
                    """, connection)
players

---
<a id='3'></a>
# **3. Analysis by country**
[Table of Contents](#0)

<a id = 'league_by_country'></a>
## **3.1. The league of each country in database**

In [ ]:
# The league of each country in database
league_by_country = pd.read_sql_query("""
                                        SELECT 
                                            Country.name country,
                                            League.name league                                            
                                        FROM League
                                            join Country
                                        WHERE League.id = Country.id;
                                      """, connection)
league_by_country

## **3.2. Sorting matches by countries**

#### **Goal: We have seen before that there are 115 columns in 'Match' table. We will generate a table that display information on all matches with following information:**
     country, league, season, stage, date, home_team, away_team, home_team_goal, away_team_goal
#### **Method: We will combine SQL queries and pandas**
#### **Note that home_team_name and away_team name are not in 'Match' table. We only have their api_id. We will need to extract names correcponding to this id from 'Team' table.**

### **We first select key columns**

In [ ]:
# select columns of interest: columns' titles
selected_matches_columns = ['id', 'country_id', 'season', 'stage', 'date', 'match_api_id', 
                            'home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal']

In [ ]:
# Display all matches. 
matches = pd.read_sql("""
                        SELECT 
                            id, country_id, season, stage, date, match_api_id, 
                            home_team_api_id, away_team_api_id, home_team_goal, away_team_goal
                        FROM 
                            Match;
                    """, connection)

In [ ]:
# Select columns of interest from dataframe.
matches[['id', 'country_id', 'season', 'stage', 'date',
       'match_api_id', 'home_team_api_id', 'away_team_api_id',
       'home_team_goal', 'away_team_goal']]

### **Our next task is to replace 'home_team_api_id' and 'away_team_api_id' with their corresponding 'team_long_name' from 'Team' table.**

In [ ]:
teams.columns

In [ ]:
matches_by_countries = pd.read_sql("""
                                    SELECT                                          --Choose columns--
                                        Country.name as country,
                                        League.name as leagea, 
                                        Match.season, 
                                        Match.stage, 
                                        Match.date,
                                        Team.team_long_name as 'home_team_name',         --Replace home_team_api_id by team_long_name-- 
                                        Match.away_team_api_id, 
                                        Match.home_team_goal, 
                                        Match.away_team_goal
                                    FROM 
                                        Country join League join Match join Team
                                    WHERE 
                                        Country.id = League.country_id and
                                        Country.id = Match.country_id and 
                                        Team.team_api_id = Match.home_team_api_id
                                        ;""", connection)
matches_by_countries                 

### **We now need to replace away_team_api_id with corresponding team name.**
* To do this, we create a new table of two columns 'team_long_name' from 'Team' table and 'away_team_api_id' from 'Match' table. We then use this table to match name of 'away_team_api_id' from the table we just created above.

In [ ]:
# generate away_team_name from Team table that matches away_team_api_id from Match table
away_team_name_and_id = pd.read_sql("""
                                        SELECT 
                                            Team.team_long_name as 'away_team_name',
                                            Match.away_team_api_id
                                        FROM 
                                            Match join Team
                                        WHERE 
                                            Team.team_api_id = Match.away_team_api_id;
                                    """, connection)
away_team_name_and_id

### **We now merge two dataframes.**

In [ ]:
matches_by_countries_raw = pd.merge(matches_by_countries, away_team_name_and_id, how = 'left', on = 'away_team_api_id').drop_duplicates()
matches_by_countries_raw


In [ ]:
# check columns before arrange
matches_by_countries_raw.columns

In [ ]:
# choose columns in order
new_columns = ['country', 'leagea', 'season', 'stage', 'date', 'home_team_name', 'away_team_name', 'home_team_goal', 'away_team_goal']
# display all matches in order of columns of interest
matches_by_countries_final = matches_by_countries_raw[new_columns]
matches_by_countries_final

### **We now add one more column showing if home_team won/lost each game**

In [ ]:
# define a function for result: win -> 1, lost -> -1, tie -> 0
def this_map(x):
    if x < 0:
        return -1
    if x > 0:
        return 1
    return 0
matches_by_countries_final['result'] = matches_by_countries_final['home_team_goal'] - matches_by_countries_final['away_team_goal']
matches_by_countries_final['result'] = matches_by_countries_final['result'].apply(lambda x:this_map(x))
matches_by_countries_final


## **3.3. Counting matches by countries and sorting by increasing order**

In [ ]:
count_matches_by_countries = pd.read_sql("""
                                            SELECT                                         
                                                Country.name country,
                                                Count() as 'number of matches'
                                            FROM 
                                                Country join Match
                                            WHERE 
                                                Country.id= Match.country_id
                                            GROUP BY Country.name
                                            ORDER BY 'number of matches';
                                        """, connection)
count_matches_by_countries   

---
<a id='4'></a>
# **4. Analyse teams**
[Table of Contents](#0)

* **In this section, we analyse relationship between number of games each team played, how many goals each team scored and lost in total, what ratio between number of goal a team could scored v.s. lost each game.**

## **4.1. Count matches by teams**

In [ ]:
# Insert column of how many games each team played during the season
matches_by_teams = matches_by_countries_final.groupby(['home_team_name'])['home_team_name'].count().reset_index(name="games_played")
matches_by_teams

## **4.2. Count goals scored by teams**

In [ ]:
# Insert column of total goals a team scored during season
goals_scored_by_teams = matches_by_countries_final.groupby(['home_team_name'])['home_team_goal'].sum().reset_index(name="goals_scored")
goals_scored_by_teams

## **4.3. Count goals lost by team**

In [ ]:
# Insert column of total goals a team lost during the season
goals_lost_by_teams = matches_by_countries_final.groupby(['home_team_name'])['away_team_goal'].sum().reset_index(name="goals_lost")
goals_lost_by_teams

## **4.4. Merge dataframes**
* **We will display number of games played by team and how many goals they scored and lost during the season.**

In [ ]:
# merge dataframe by home_team_name
goals_by_teams = pd.merge(pd.merge(matches_by_teams,goals_scored_by_teams,on='home_team_name'),goals_lost_by_teams,on='home_team_name')
goals_by_teams

## **4.5. Count goals scored and lost per game by teams**

In [ ]:
# add column of goals scored per game during the season
goals_by_teams['goals_scored_per_game'] = goals_by_teams['goals_scored']/goals_by_teams['games_played']
goals_by_teams

In [ ]:
# add column of goals lost per game during the season
goals_by_teams['goals_lost_per_game'] = goals_by_teams['goals_lost']/goals_by_teams['games_played']
goals_by_teams

## **4.6. Ratio of goals scored v.s lost per game by teams**

In [ ]:
# add column of ratio of goals scored/goals lost during the season 
goals_by_teams['goal_scored/goal_lost ratio'] = goals_by_teams['goals_scored']/goals_by_teams['goals_lost']
goals_by_teams

## **4.7. Visualise distribution of goals**

#### **Distribution of goals_scored_per_game**

In [ ]:
sns.displot(data=goals_by_teams,x='goals_scored_per_game', kde=True)

#### **Distribution of goals_lost_per_game**

In [ ]:
sns.displot(data=goals_by_teams,x='goals_lost_per_game', kde=True)

#### **Distribution of goals_scored/goal_lost_ ratio**


In [ ]:
sns.displot(data=goals_by_teams,x='goal_scored/goal_lost ratio', kde=True)

___
<a id ='5'></a>
# **5. Analyse win and lose by teams**
[Table of Contents](#0)

## **5.1. Count number of games a team won during the season**

In [ ]:
# Count number of games a team won during the season
filter_won_results = matches_by_countries_final[matches_by_countries_final['result']== 1]
won_games_by_teams = filter_won_results.groupby('home_team_name')['result'].count().reset_index(name = "matches_won")
won_games_by_teams

## **5.2. Count number of games a team lost during the season**

In [ ]:
# Count number of games a team lost during the season
filter_lost_results = matches_by_countries_final[matches_by_countries_final['result']== -1]
lost_games_by_teams = filter_lost_results.groupby('home_team_name')['result'].count().reset_index(name = "matches_lost")
lost_games_by_teams

## **5.3.Count number of games a team tie during the season**

In [ ]:
# Count number of games a team tie during the season
filter_tie_results = matches_by_countries_final[matches_by_countries_final['result']== 0]
tie_games_by_teams = filter_tie_results.groupby('home_team_name')['result'].count().reset_index(name = "matches_tie")
tie_games_by_teams

## **5.4. Merge dataframes**

In [ ]:
# merge dataframes
result_by_teams = pd.merge(pd.merge(pd.merge(matches_by_teams,won_games_by_teams,on='home_team_name'),
                                        lost_games_by_teams,on='home_team_name'), 
                                        tie_games_by_teams, on='home_team_name')
result_by_teams

#### Let's add three columns: won/game, lost/game, and tie/game

In [ ]:
result_by_teams['won/game ratio'] = result_by_teams['matches_won']/result_by_teams['games_played']
result_by_teams['lost/game ratio'] = result_by_teams['matches_lost']/result_by_teams['games_played']
result_by_teams['tie/game ratio'] = result_by_teams['matches_tie']/result_by_teams['games_played']
result_by_teams

## **5.6. Visualise distribution of ratios**

In [ ]:
# Distribution of won/game ratio
sns.displot(data=result_by_teams,x='won/game ratio', kde=True)

In [ ]:
# Distribution of lost/game ratio
sns.displot(data=result_by_teams,x='lost/game ratio', kde=True)

In [ ]:
# Distribution of tie/game ratio
sns.displot(data=result_by_teams,x='tie/game ratio', kde=True)

---
---
## **Thanks for your interest and I really appreciate your comments/suggestions!**